In [2]:
import json
from datetime import datetime
import plotly.express as px
from sklearn.model_selection import train_test_split
import sqlite3
import nltk
import spacy
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import umap
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
%matplotlib inline

In [24]:
df_reviews_4yrs_open = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_reviews_last_4yrs_open.csv')



/Users/robholmstrom/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
df_business_4yrs_open = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_business_last_4yrs_open.csv')


In [30]:
df_business_4yrs_open['review_count'].value_counts()

3       25906
4       14132
5       10986
6        8852
7        7430
        ...  
1445        1
2648        1
3414        1
2855        1
1791        1
Name: review_count, Length: 1292, dtype: int64

In [31]:
df_bus_select =  df_business_4yrs_open[df_business_4yrs_open['review_count']>10]

In [32]:
df_bus_select['review_count'].value_counts()

11      4114
12      3555
13      3232
14      2893
15      2707
        ... 
1204       1
1268       1
2164       1
2228       1
1791       1
Name: review_count, Length: 1284, dtype: int64

In [33]:
threshold = 0.1
review_by_state = pd.DataFrame(df_bus_select['state'].value_counts())
states_select = review_by_state[review_by_state['state'] > threshold*review_by_state.max()['state']]
states_select

,state
AZ,25777
NV,18208
ON,11548
NC,6122
OH,5436
PA,4298
QC,3020


In [34]:
df_reviews_4yrs_open.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464626 entries, 0 to 4464625
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Unnamed: 0   object 
 1   review_id    object 
 2   user_id      object 
 3   business_id  object 
 4   stars        float64
 5   useful       float64
 6   funny        float64
 7   cool         float64
 8   text         object 
 9   date         object 
 10  year         float64
dtypes: float64(5), object(6)
memory usage: 374.7+ MB


In [36]:
df_rev_bus_raw = df_reviews_4yrs_open.merge(df_bus_select, on = 'business_id', how= 'inner')

In [37]:
df_rev_bus_raw.shape

(4135344, 25)

In [38]:
df_rev_bus_raw = df_rev_bus_raw.rename(columns = {'stars_x': 'stars'})

In [39]:
df_rev_bus_select = df_rev_bus_raw.loc[df_rev_bus_raw['state'].isin(states_select.index)==True]

In [40]:
df_rev_bus_select.state.value_counts()

NV    1512272
AZ    1335777
ON     414192
NC     276269
OH     186370
PA     175134
QC      97686
Name: state, dtype: int64

In [41]:
df_food_rev = df_rev_bus_select[df_rev_bus_select['categories'].str.contains('Restaurant')|
                 df_rev_bus_select['categories'].str.contains('Coffee & Tea')]

In [42]:
df_food = df_food_rev.copy()
catlist_clean = df_food['categories'].str.replace(' ', '' ).str.replace(',', ', ')

In [43]:
df_food['categories_filtered']= catlist_clean

In [44]:
food_categories = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/Yelp_food_categories.csv')
food_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Categories   0 non-null      float64
 1   Food_types   69 non-null     object 
 2   Ethnicities  96 non-null     object 
 3   Venue_type   51 non-null     object 
dtypes: float64(1), object(3)
memory usage: 6.9+ KB


In [45]:
food_types = food_categories['Food_types'].dropna().str.strip().str.replace(' ', '')
ethnicities = food_categories['Ethnicities'].dropna().str.strip().str.replace(' ', '')
venue_types = food_categories['Venue_type'].dropna().str.strip().str.replace(' ', '')

In [46]:
type_list = []

for x in df_food['categories_filtered']:
    sublist = []
    for i in food_types:
        if i in x:
            sublist.append(i)
    
    if len(sublist) == 0:
        type_list.append(['Undefined'])
    else:
        type_list.append(sublist)  

print(len(type_list))
print(df_food.shape[0])


2638450
2638450


In [47]:
venue_list = []

for x in  df_food['categories_filtered']:
    sublist = []
    for i in venue_types:
        if i in x:
            sublist.append(i)
    
    if len(sublist) == 0:
        venue_list.append(['Restaurant'])
    else:
        venue_list.append(sublist)  

print(len(venue_list))
print(df_food.shape[0])


2638450
2638450


In [48]:
ethnicity_list = []

for x in  df_food['categories_filtered']:
    sublist = []
    for i in ethnicities:
        if i in x:
            sublist.append(i)
    
    if len(sublist) == 0:
        ethnicity_list.append(['Undefined'])
    else:
        ethnicity_list.append(sublist)  
print(len(ethnicity_list))
print(df_food.shape[0])


2638450
2638450


In [49]:
df_food['food_type'] = type_list
df_food['ethnicity'] = ethnicity_list
df_food['venue'] = venue_list


In [50]:
df_food['food_comb'] = (df_food['food_type']+df_food['ethnicity']+df_food['venue']).apply(lambda x: ', '.join(x))
df_food['food_comb'] = df_food['food_comb'].str.replace('Undefined,', '')


In [51]:
df_food_clean = df_food.drop(columns = ['categories'])
df_food_clean['food_type'] = df_food_clean['food_type'].apply(lambda x: ', '.join(x))
df_food_clean['venue'] = df_food_clean['venue'].apply(lambda x: ', '.join(x))
df_food_clean['ethnicity']= df_food_clean['ethnicity'].apply(lambda x: ', '.join(x))

In [52]:
(df_food_clean['postal_code']=='').value_counts()

False    2638450
Name: postal_code, dtype: int64

In [53]:
df_food_clean['postal_code'].replace('', np.nan, inplace = True)
df_food_clean.dropna(inplace = True)

In [54]:
df_food_clean.reset_index(drop=True, inplace= True)


In [55]:
df_food_clean = df_food_clean.rename(columns = {'stars_x':'stars'})

In [56]:
df_food_clean

,Unnamed: 0_x,review_id,user_id,business_id,stars,useful,funny,cool,text,date,...,stars_y,review_count,is_open,attributes,hours,categories_filtered,food_type,ethnicity,venue,food_comb
0,4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0.0,0.0,0.0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
1,3103,gQuXBrYo8dqY7h_rfDzwLw,46TNf-5T5EcNFRJPYv_nrw,IS4cv902ykd8wj1TR0N3-A,4.0,0.0,0.0,0.0,Love the chicken. It's always hot and fresh ta...,2018-01-09 20:25:20,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
2,3173,xkq0ekkZ39uSbb29LihkfQ,OTGWjBCELxkZ6lpS2OEtxQ,IS4cv902ykd8wj1TR0N3-A,3.0,0.0,1.0,0.0,"Great food right down the street, long line si...",2017-01-28 00:03:26,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
3,20520,m4AkgxXvb-GHuDK4b9i9sA,YvlcRfeulgtusMFBOQTWzw,IS4cv902ykd8wj1TR0N3-A,1.0,0.0,0.0,1.0,Terrible terrible service I waited 30 minutes ...,2017-01-15 05:55:07,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
4,22003,OkgQVoHBJ6w-4qL-9lOczA,DBq1YkbIJ4yTXUYJw1V_KA,IS4cv902ykd8wj1TR0N3-A,3.0,0.0,0.0,0.0,I am giving it 3 stars because all the drinks ...,2017-03-15 21:36:27,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569382,7962856,zgdS5ggJ4D6Eers3VyXaQQ,8MYvRl2qr3Ys6URmLSLJDw,KzNG3-kfGCC01USMS2BZvA,4.0,0.0,1.0,0.0,Great wings! Food is good staff is friendly an...,2017-07-02 00:39:31,...,3.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...","Restaurants, Pizza",Pizza,Undefined,Restaurant,"Pizza, Restaurant"
2569383,7962857,BoACFPqsa_wLvG61ZYJqTg,RuxZ18eNzPRU2rpVvzYGkA,KzNG3-kfGCC01USMS2BZvA,1.0,2.0,0.0,0.0,"Wings were good, delivery drivers with Nazi ta...",2018-08-07 13:41:56,...,3.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...","Restaurants, Pizza",Pizza,Undefined,Restaurant,"Pizza, Restaurant"
2569384,7971875,KWV3NOmZgnqmfns-bG1PVA,7K0kU03q2kWGD4I1ZsMznA,KzNG3-kfGCC01USMS2BZvA,5.0,1.0,0.0,0.0,Love the wings from the West Mifflin location....,2016-09-30 00:29:32,...,3.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...","Restaurants, Pizza",Pizza,Undefined,Restaurant,"Pizza, Restaurant"
2569385,8011272,MKNsjnas-oh64zx0LhcopQ,HtuReE1QJa2Catuq4Z0z9g,KzNG3-kfGCC01USMS2BZvA,5.0,0.0,1.0,1.0,"Best wing sauces in the city, great subs fresh...",2016-07-21 22:47:06,...,3.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...","Restaurants, Pizza",Pizza,Undefined,Restaurant,"Pizza, Restaurant"


In [57]:
df_food_clean.drop(['Unnamed: 0_x'], axis = True, inplace = True)

In [58]:
df_food_clean.to_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_food_clean_rev.csv')

In [59]:
df_food_clean['review_count'].value_counts()

92     7426
75     7343
50     7328
49     7263
44     7225
       ... 
500     266
602     247
609     243
777     235
652     184
Name: review_count, Length: 1222, dtype: int64